In [2]:
from skyfield.api import load, N, W, wgs84, utc, Angle
from datetime import datetime
ts = load.timescale()
# t = ts.from_datetime(datetime.strptime("2024-04-08 19:13 UTC", "%Y-%m-%d %H:%M %Z").replace(tzinfo=utc))
de421 = load("de421.bsp")
earth, sun, moon = de421["earth"], de421["sun"], de421["moon"]
caledonia_wsg = wgs84.latlon(40.6318188 * N, 82.9766802 * W)
caledonia = earth + caledonia_wsg

In [3]:
def print_observe(from_obj, obj):
    pos = from_obj.at(t).observe(obj)
    alt, az, d = pos.apparent().altaz()
    ra, dec, d = pos.radec()
    print(f"{obj.target_name.split()[-1]}\nalt: {alt.degrees:.2f}\t az: {az.degrees:.1f}\n ra: {ra.hours:.1f}h\tdec: {dec.degrees:.1f}")

In [4]:
t = ts.now()
lst = caledonia_wsg.lst_hours_at(t)
print(f"{t.tt_strftime()}\nLST: {Angle(hours=lst)} ({lst:.2f}h)")
print_observe(caledonia, sun)
print_observe(caledonia, moon)

2025-03-14 00:29:01 TT
LST: 06h 23m 30.46s (6.39h)
SUN
alt: -10.49	 az: 275.7
 ra: 23.6h	dec: -2.7
MOON
alt: 12.63	 az: 96.1
 ra: 11.5h	dec: 3.8
